<a href="https://colab.research.google.com/github/nachochipzz/Project-Submission/blob/main/n213-regularized-regression/n213a_regularized_regression_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 1 / NOTE 3*

# 📝 Assignment
---

# 🌱 **기본 과제**

이번에는 노트에서 다루지 않은 새로운 데이터인 [Melbourne Housing Market](https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market#:~:text=calendar_view_week-,MELBOURNE_HOUSE_PRICES_LESS,-.csv)을 사용하여 정규화를 연습해 보겠습니다.

In [130]:
!pip install category_encoders
!pip install scikit-learn==1.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Part 1. 데이터 준비**

### **1-1. 데이터 불러오기 및 전처리**

- 데이터셋을 불러오세요.
- 다음 항목을 제거하세요.
  - column
    - 날짜 특성
    - 범주형 데이터 중에서 카디널리티가 50이 넘는 특성
  - row
    - Price 변수 기준 결측치 샘플
    - 중복되는 샘플 

#### **Q1. 다음 column 중 범주형 데이터가 아닌 것을 고르세요.**
```
A. Address 
B. Rooms 
C. SellerG 
D. Postcode 
E. CouncilArea 
```
- 답 : B. Rooms (수치형 자료)

In [131]:
import pandas as pd

# get data
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/melbourne_house_prices/MELBOURNE_HOUSE_PRICES_LESS.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,"1,490,000.00",S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.00,Yarra City Council
1,Abbotsford,59A Turner St,3,h,"1,220,000.00",S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.00,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,"1,420,000.00",S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.00,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,"1,515,000.00",S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.50,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,"670,000.00",S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.40,Moonee Valley City Council


In [132]:
# 날짜 특성 drop
df.drop(columns='Date', inplace=True)

In [133]:
# 범주형 데이터 중 카디널리티가 50이 넘는 특성 확인
df.nunique() #Suburb, Address, SellerG, Postcode 칼럼 drop

Suburb             380
Address          57754
Rooms               14
Type                 3
Price             3417
Method               9
SellerG            476
Postcode           225
Regionname           8
Propertycount      368
Distance           180
CouncilArea         34
dtype: int64

In [134]:
# 범주형 데이터 중 카디널리티가 50이 넘는 특성 drop
df.drop(columns=['Suburb', 'Address', 'SellerG', 'Postcode'], inplace=True)
df.head()

,Rooms,Type,Price,Method,Regionname,Propertycount,Distance,CouncilArea
0,3,h,"1,490,000.00",S,Northern Metropolitan,4019,3.00,Yarra City Council
1,3,h,"1,220,000.00",S,Northern Metropolitan,4019,3.00,Yarra City Council
2,3,h,"1,420,000.00",S,Northern Metropolitan,4019,3.00,Yarra City Council
3,3,h,"1,515,000.00",S,Western Metropolitan,1543,7.50,Moonee Valley City Council
4,2,h,"670,000.00",S,Western Metropolitan,3464,10.40,Moonee Valley City Council


In [135]:
# Price 변수 기준 결측치 샘플 drop
df.dropna(subset=['Price'], inplace=True)

In [136]:
# 중복되는 샘플 drop
df.drop_duplicates(inplace=True)

### **1-2. Scaling & Encoding**
- 데이터 분리
  - 특성 X와 타겟 y를 지정하세요. 
  - `train_test_split()`을 사용하여 전체 데이터를 훈련과 테스트 데이터로 나누어주세요. 
    - 훈련과 테스트 데이터는 8:2의 비로 나눕니다.
    - `random_state=2`로 설정합니다. 
  - `train_test_split()`을 사용하여 훈련 데이터를 훈련과 검증 데이터로 나누어주세요.
    - 훈련과 검증 데이터는 8:2의 비로 나눕니다.
    - `random_state=2`로 설정합니다.
- 수치형 특성에 대해 **`StandardScaler()`**를 사용해서 **표준화**를 수행하세요.
- 범주형 특성에 대해 `category_encoders` 라이브러리의 **`OneHotEncoder()`**를 사용해서 원핫인코딩을 수행하세요.

In [137]:
df.head()

,Rooms,Type,Price,Method,Regionname,Propertycount,Distance,CouncilArea
0,3,h,"1,490,000.00",S,Northern Metropolitan,4019,3.00,Yarra City Council
1,3,h,"1,220,000.00",S,Northern Metropolitan,4019,3.00,Yarra City Council
2,3,h,"1,420,000.00",S,Northern Metropolitan,4019,3.00,Yarra City Council
3,3,h,"1,515,000.00",S,Western Metropolitan,1543,7.50,Moonee Valley City Council
4,2,h,"670,000.00",S,Western Metropolitan,3464,10.40,Moonee Valley City Council


In [138]:
# 데이터 분리

# 함수 불러오기
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# X, y 지정
target = 'Price'
X = df.drop(target, axis =1)
y = df[target]

# 학습용, 검증용, 평가용 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2)

In [139]:
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import warnings
warnings.filterwarnings(action='ignore')

In [140]:
# 수치형 특성 표준화
numeric_feats = X_train.dtypes[X_train.dtypes != "object"].index

scaler = StandardScaler()
X_train[numeric_feats] = scaler.fit_transform(X_train[numeric_feats])
X_val[numeric_feats] = scaler.transform(X_val[numeric_feats])
X_test[numeric_feats] = scaler.transform(X_test[numeric_feats])

##### **Q2. 범주형 특성에 원핫인코딩을 수행한 후, 열(column) 개수를 제출하세요.**

In [141]:
# One-Hot encoding 불러오고 선언
from category_encoders import OneHotEncoder
ohe = OneHotEncoder()

# 인코딩 수행
X_train_ohe = ohe.fit_transform(X_train)
X_test_ohe = ohe.transform(X_test)
X_val_ohe = ohe.transform(X_val)

In [142]:
(X_train_ohe.dtypes == 'object').sum()

0

In [143]:
X_train_ohe.shape[1]

53

## **Part 2. 정규화 모델**

- 기준모델
  - train set에 대해 평균을 사용하여 기준모델을 만드세요.
  - $R^2$와 MAE 값을 확인하세요.
- **`ols`** vs **`ridge`** vs **`lasso`**
  -  일반(다중) 회귀와 정규화 회귀 모델을 각각 아래와 같은 조건에서 만드세요. 
    - `LinearRegression()`
    - `Ridge()`
      - 적절한 alpha 값을 설정하세요.
    - `Lasso()`
      - 적절한 alpha 값을 설정하세요.
  - 각 모델의 $R^2$와 MAE 값을 확인하세요.
  - 각 모델의 회귀계수를 시각화로 나타내 보세요.

In [144]:
# 소수점 보여주는 정도 설정
pd.options.display.float_format = '{:,.2f}'.format

In [145]:
# 기준 모델 설정
base = [y_train.mean()]*len(y_train)

In [146]:
# 모델 평가 지수 반환하는 함수 설정
from sklearn.metrics import r2_score, mean_absolute_error

def scores(y_true, y_pred):
   mae = mean_absolute_error(y_true, y_pred)
   r2 = r2_score(y_true, y_pred)
   return mae, r2

In [147]:
# 기준 모델 mae, r2
base_mae, base_r2 = scores(y_train, base)
base_mae, base_r2

(420382.30671549414, 0.0)

In [148]:
from sklearn.model_selection import cross_val_score

# 일반 다중 선형 회귀 선언
ols = LinearRegression()

# 모델 학습
ols.fit(X_train_ohe, y_train)
ols_pred_tr = ols.predict(X_train_ohe)
ols_pred_val = ols.predict(X_val_ohe)

# 성능 비교
ols_trainscore_mae, ols_trainscore_r2 = scores(y_train, ols_pred_tr)
ols_valscore_mae, ols_valscore_r2 = scores(y_val, ols_pred_val)
print('OLS train MAE :', ols_trainscore_mae)
print('OLS train R2:', ols_trainscore_r2 )
print('OLS val MAE :', ols_valscore_mae)
print('OLS val R2:', ols_valscore_r2)

OLS train MAE : 251432.71777720607
OLS train R2: 0.5787266918673125
OLS val MAE : 260252.4356901367
OLS val R2: 0.5409560596346819


### **2-1. Ridge Regression**

##### **Q3. Ridge 회귀 모델에서 절댓값이 가장 큰 회귀계수 값은 얼마인가요? (소수점 이하 반올림)**

In [149]:
# ridge regression 구현

from sklearn.linear_model import RidgeCV, LassoCV

# Ridge 회귀 모델 선언
alphas=[0.1, 1, 10 ,100]
ridge = RidgeCV(alphas=alphas, cv=5)

# Ridge 모델 학습
ridge.fit(X_train_ohe, y_train)
ridge_pred_tr = ridge.predict(X_train_ohe)
ridge_pred_val = ridge.predict(X_val_ohe)

# 성능 비교
ridge_trainscore_mae, ridge_trainscore_r2 = scores(y_train, ridge_pred_tr)
ridge_valscore_mae, ridge_valscore_r2 = scores(y_val, ridge_pred_val)
print('Ridge train MAE :', ridge_trainscore_mae)
print('Ridge train R2:', ridge_trainscore_r2 )
print('Ridge val MAE :', ridge_valscore_mae)
print('Ridge val R2:', ridge_valscore_r2)

Ridge train MAE : 251399.5432471496
Ridge train R2: 0.5787175201337521
Ridge val MAE : 260237.28510519816
Ridge val R2: 0.5409041369051361


In [170]:
comparison = pd.DataFrame(index=['mae', 'r2'], columns=['Base', 'OLS train','OLS Validation'])
comparison['Base'] = scores(y_train, base)
comparison['OLS train'] = scores(y_train, ols_pred_tr)
comparison['OLS Validation'] = scores(y_val, ols_pred_val)
comparison

,Base,OLS train,OLS Validation
mae,"420,382.31","251,432.72","260,252.44"
r2,0.00,0.58,0.54


In [171]:
comparison['Ridge train'] = scores(y_train, ridge_pred_tr)
comparison['Ridge Validation'] = scores(y_val, ridge_pred_val)
comparison

,Base,OLS train,OLS Validation,Ridge train,Ridge Validation
mae,"420,382.31","251,432.72","260,252.44","251,399.54","260,237.29"
r2,0.00,0.58,0.54,0.58,0.54


In [154]:
print('\nbest alpha')
print(ridge.alpha_)

print('\n회귀계수')
print(pd.Series(ridge.coef_).nlargest(10))


best alpha
1.0

회귀계수
32   494,146.80
30   460,170.82
20   451,668.68
50   390,015.93
42   296,846.29
52   240,757.88
0    234,784.80
1    227,777.42
49   226,681.80
24   184,787.65
dtype: float64


### **2-2. Lasso Regression**

###### **Q4. Lasso 회귀 모델에서 회귀계수 값이 0으로 수렴한 특성은 모두 몇 개인가요?**

In [153]:
# lasso regression 구현

# lasso 회귀 모델 선언
lasso = LassoCV(alphas=alphas, cv=5)

# Ridge 모델 학습
lasso.fit(X_train_ohe, y_train)
lasso_pred_tr = lasso.predict(X_train_ohe)
lasso_pred_val = lasso.predict(X_val_ohe)

# 성능 비교
lasso_trainscore_mae, lasso_trainscore_r2 = scores(y_train, lasso_pred_tr)
lasso_valscore_mae, lasso_valscore_r2 = scores(y_val, lasso_pred_val)
print('Lasso train MAE :', lasso_trainscore_mae)
print('Lasso train R2:', lasso_trainscore_r2 )
print('Lasso val MAE :', lasso_valscore_mae)
print('Lasso val R2:', lasso_valscore_r2)

Lasso train MAE : 251392.1858988851
Lasso train R2: 0.5787152349119042
Lasso val MAE : 260232.2133099389
Lasso val R2: 0.540908326259663


In [172]:
comparison['Lasso train'] = scores(y_train, lasso_pred_tr)
comparison['Lasso Validation'] = scores(y_val, lasso_pred_val)
comparison.T

,mae,r2
Base,"420,382.31",0.00
OLS train,"251,432.72",0.58
OLS Validation,"260,252.44",0.54
Ridge train,"251,399.54",0.58
Ridge Validation,"260,237.29",0.54
Lasso train,"251,392.19",0.58
Lasso Validation,"260,232.21",0.54


In [156]:
print('\nbest alpha')
print(lasso.alpha_)

print('\n회귀계수')
print(pd.Series(lasso.coef_).sort_values())


best alpha
10.0

회귀계수
41   -385,696.41
44   -353,833.14
23   -347,155.91
47   -280,175.97
36   -240,999.72
18   -238,241.17
25   -220,826.52
19   -181,740.36
43   -166,760.08
26   -165,875.29
9    -153,195.30
34    -87,975.76
12    -86,713.82
3     -78,158.03
33    -66,505.13
40    -53,923.17
29    -40,957.49
22    -36,086.10
6     -33,039.46
35    -29,441.71
13    -10,906.76
10     -3,941.94
8      -2,812.39
37         -0.00
45          0.00
46         -0.00
51          0.00
5          -0.00
17        640.27
7      23,523.65
4      23,874.50
14     26,487.64
38     67,087.31
48     67,179.73
28     68,133.18
11     98,484.78
2     115,020.64
27    145,296.17
21    164,506.82
15    172,892.26
16    177,672.38
39    180,376.15
31    223,067.25
24    225,709.06
0     234,769.58
52    247,613.17
49    269,439.84
42    337,699.88
1     360,151.71
50    430,209.85
20    492,997.47
30    501,286.57
32    535,719.77
dtype: float64


### **2-3. 일반화 성능 검증**
-  학습한 정규화 모델 중 검증 세트의 **`MAE`**가 가장 낮은 모델을 선택하고 일반화 성능을 확인하세요.
  - 일반화 성능은 **`Test set`**를 한 번만 사용해서 확인합니다.
  - **$R^2$**와 **MAE** score를 확인합니다.

In [173]:
comparison.drop(columns=['OLS Validation','OLS train', 'Ridge train', 'Lasso train'], inplace=True)

In [174]:
comparison.T.sort_values('mae')

,mae,r2
Lasso Validation,"260,232.21",0.54
Ridge Validation,"260,237.29",0.54
Base,"420,382.31",0.00


In [177]:
# mae가 가장 낮은 규제 모델은 Lasso 회귀 모델
lasso_pred_test = lasso.predict(X_test_ohe)
lasso_testscore_mae, lass_testscore_r2 = scores(y_test, lasso_pred_test)
print()


# 성능 비교
lasso_trainscore_mae, lasso_trainscore_r2 = scores(y_train, lasso_pred_tr)
lasso_valscore_mae, lasso_valscore_r2 = scores(y_val, lasso_pred_val)
comparison['Lasso test'] = scores(y_train, lasso_pred_tr)
display(comparison)

,Base,Ridge Validation,Lasso Validation,Lasso test
mae,"420,382.31","260,237.29","260,232.21","251,392.19"
r2,0.00,0.54,0.54,0.58


###### **Q5. 정규화 모델 중 검증 세트의 MAE가 가장 낮은 모델의 Test set에 대한 MAE score를 제출하세요. (소수점 이하 반올림)** 

답 : 260,232

##### **Q6. 최종모델은 기준모델보다 개선된 모델인가요? (test set에서의 $R^2$와 MAE를 기준모델과 비교해보세요.)**
```
A. test set에서의 R2가 기준모델보다 감소한 것을 보아 성능이 더 좋은 모델이다.
B. test set에서의 MAE가 기준모델보다 증가한 것을 보아 성능이 더 좋은 모델이다.
C. test set에서의 R2가 증가하고 MAE가 감소한 것을 보아 성능이 더 좋은 모델이다.
D. test set에서의 MAE와 R2가 기준모델보다 증가한 것을 보아 성능이 더 좋은 모델이다.
```
- 답 : C. test set에서의 R2가 증가하고 MAE가 감소한 것을 보아 성능이 더 좋은 모델이다.

# 🥇 **도전 과제**

- RidgeCV와 LassoCV를 사용해서 최적의 알파값을 찾는 과정을 수행하고 성능을 개선해보세요.

- ElasticNet 정규화 모델에 대해서 추가로 학습해보세요. 

- Lasso 와 Ridge에 사용되는 L1, L2 정규화에 대해 알아보고 논의해보세요.
